In [1]:
# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import random

### Read-in the emails and print some basic statistics

Nsamp = 5000
max_cells = 50
maxlen = 20

# Install Simon
#!pip install git+https://github.com/algorine/simon
#from Simon.LengthStandardizer import DataLengthStandardizerRaw

# Enron
EnronEmails = pd.read_csv('../input/enron-email-bodies/enron_emails_body.csv',dtype='str', header=0)
print("The size of the Enron emails dataframe is:")
print(EnronEmails.shape)
print("Ten Enron emails are:")
print(EnronEmails.loc[:10])

# Spam
SpamEmails = pd.read_csv('../input/fraudulent-email-bodies/fraudulent_emails_body.csv',encoding="ISO-8859-1",dtype='str', header=0)
print("The size of the Spam emails dataframe is:")
print(SpamEmails.shape)
print("Ten Spam emails are:")
print(SpamEmails.loc[:10])

# Convert everything to lower-case, put one sentence per column in a tabular
# structure, truncate to max_cells...
ProcessedEnronEmails=[row.lower().split('\n')[:max_cells] for row in EnronEmails.iloc[:,1]]
#print("3 Enron emails after Processing (in list form) are:")
#print((ProcessedEnronEmails[:3]))
EnronEmails = pd.DataFrame(random.sample(ProcessedEnronEmails,Nsamp)).transpose()


#EnronEmails = DataLengthStandardizerRaw(EnronEmails,max_cells)


#print("Ten Enron emails after Processing (in DataFrame form) are:")
#print((EnronEmails[:10]))
print("Enron email dataframe after Processing shape:")
print(EnronEmails.shape)

ProcessedSpamEmails=[row.lower().split('/n')[:max_cells] for row in SpamEmails.iloc[:,1]]
#print("3 Spam emails after Processing (in list form) are:")
#print((ProcessedSpamEmails[:3]))
SpamEmails = pd.DataFrame(random.sample(ProcessedSpamEmails,Nsamp)).transpose()


#SpamEmails = DataLengthStandardizerRaw(SpamEmails,max_cells)


#print("Ten Spam emails after Processing (in DataFrame form) are:")
#print((SpamEmails[:10]))
print("Spam email dataframe after Processing shape:")
print(SpamEmails.shape)

The size of the Enron emails dataframe is:
(20000, 2)
Ten Enron emails are:
   Unnamed: 0                                                  0
0           0                          here is our forecast\n\n 
1           1  traveling to have a business meeting takes the...
2           2                     test successful.  way to go!!!
3           3  randy,\n\n can you send me a schedule of the s...
4           4                let's shoot for tuesday at 11:45.  
5           5  greg,\n\n how about either next tuesday or thu...
6           6  please cc the following distribution list with...
7           7                   any morning between 10 and 11:30
8           8  1. login:  pallen pw: ke9davis\n\n i don't thi...
9           9  ---------------------- forwarded by phillip k ...
10         10  mr. buckner,\n\n for delivered gas behind san ...
The size of the Spam emails dataframe is:
(5187, 2)
Ten Spam emails are:
   Unnamed: 0                                               text
0     

In [3]:
import numpy as np

raw_data = np.column_stack((SpamEmails,EnronEmails)).T
print("DEBUG::raw_data:")
print(raw_data.shape)

# corresponding labels
Categories = ['spam','notspam']
header = ([0]*Nsamp)
header.extend(([1]*Nsamp))

DEBUG::raw_data:
(10000, 50)


In [4]:
import os

# function for processing data into the right format
def load_data(raw_data,header):
    indices, labels = [], []
    for i in range(raw_data.shape[0]):
        out=''
        for text in raw_data[i,:]:
            out = str(text)[:maxlen]+out
        out = ' '.join(out.split()[0:150])
        indices.append(out)
        labels.append(header[i])
        #print(i)
    indices = np.array(indices, dtype=object)[:, np.newaxis]
    items = list(zip(indices, labels))
    np.random.shuffle(items)
    indices, labels = zip(*items)
    indices = np.array(indices)
    return indices, np.array(labels)


# shuffle raw data first
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(b))
    data = a[p,:]
    header = np.asarray(b)[p]
    return data, list(header)

raw_data, header = unison_shuffled_copies(raw_data, header)


idx = int(0.9*raw_data.shape[0])
train_x, train_y = load_data(raw_data[:idx,:],header[:idx]) # 90% of data for training
test_x, test_y = load_data(raw_data[idx:,:],header[idx:]) # remaining 10% for testing

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x)
print(train_y[:5])
print(train_y.shape)

train_x/train_y list details, to make sure it is of the right form:
9000
[['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNonecover this. which has taken placrunning the collectientire universe of gnon-eol trades. whatbooks in which eol areporting each nightcalculations that muthere has apparentlysubject: risk book pcc: bob shults/hou/eeric bass/hou/ect@ectony harris/hou/ect@gossett/hou/ect@ect,to: david oliver/hou12/13/99 07:45 pmtorrey moorer----------------------------------------']
 ["NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNonegame tonight at 7, isubject: let's get icc: crook/corp/enron@enrmatthew lenhart/hou/baumbach/hou/ect@ectto: o'neal d winfree 08/09/2000 07:52 am from: bryan hull you know that both o"]
 ["NoneNoneNoneNoneNoneNoneNone713.306.2508=20mg=20let me know if you hstakes, and make somto 2 millions dollar down the road. frommake it work, or usentin

In [5]:
# Create datasets (Only take up to 150 words for memory)
train_text = train_x
train_label = train_y

test_text = test_x
test_label = test_y

In [6]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [7]:
# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(2, activation='softmax')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
  model.summary()
  
  return model

In [8]:
# Build and fit
model = build_model()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=5,
          batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 262,918
Trainable params: 262,918
Non-trainable params: 0
_________________________________________________________________
Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 [==============================] - 141s 16ms/step - loss: 0.3941 - sparse_categorical_accuracy: 0.8716 - val_loss: 0.1541 - val_sparse_categorical_accuracy: 0.9420
Epoch 2/5
9000/9000 [===============

In [9]:
model.save('ElmoModel.h5')
# Accuracy
preds = model.predict(test_text).argmax(axis=-1)
#print(test_label)
print(np.sum(test_label == preds) / test_label.shape[0])

0.96
